# Assignment | Article Recommendation 
### By Vikas Gupta

#### Import the required libs

In [1]:
import pandas as pd
import numpy as np

#### Load the dataset

In [2]:
df = pd.read_csv('train-201102-092914.csv')
df.head()

,user_id,article_id,rating
0,1,456,1
1,1,2934,1
2,1,82,1
3,1,1365,1
4,1,221,1


## Some EDA

In [3]:
# Size
print(f'Total Number of {df.shape[0]} records and {df.shape[1]} variables present in the dataset')

Total Number of 16731 records and 3 variables present in the dataset


In [4]:
# data types
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16731 entries, 0 to 16730
Data columns (total 3 columns):
user_id       16731 non-null int64
article_id    16731 non-null int64
rating        16731 non-null int64
dtypes: int64(3)
memory usage: 392.3 KB


###### All are integer and not null

In [5]:
df.describe()

,user_id,article_id,rating
count,16731.000000,16731.000000,16731.000000
mean,540.358616,1479.751838,1.453948
std,304.192076,860.891660,0.992538
min,1.000000,1.000000,1.000000
25%,295.000000,728.000000,1.000000
50%,533.000000,1467.000000,1.000000
75%,782.000000,2229.500000,1.000000
max,1087.000000,2976.000000,5.000000


In [6]:
# missing data
df.isnull().sum()


user_id       0
article_id    0
rating        0
dtype: int64

###### No culmuns has null values

In [7]:
# check for the distinct value for each variables

for col in df.columns:
    print(f'"{col}" has {df[col].nunique()} distinct data')


"user_id" has 907 distinct data
"article_id" has 2529 distinct data
"rating" has 5 distinct data


In [8]:
# Top 10 users who did most rating
df.groupby('user_id').count()['rating'].sort_values(ascending=False)[:10]

user_id
698     146
503     134
525     134
423     123
14      116
384     110
563     102
678      99
612      96
1038     94
Name: rating, dtype: int64

In [9]:
# Top 10 rated articles
df.groupby('article_id').count()['rating'].sort_values(ascending=False)[:10]

article_id
967     122
1425     91
467      90
2660     73
1562     71
456      69
221      64
911      61
1904     61
1433     61
Name: rating, dtype: int64

## Assignment Questions:
### Q1. Which are the top 10 articles based on a rating provided by more than 5% of users in the dataset?

In [10]:
# Five percent of users
user_5per = df['user_id'].nunique()*0.05

# filtered dataframe where only those record presnt in which atleast 5% of user rated
filtered_df = df.groupby(['article_id']).filter(lambda x: len(x) > user_5per)

# Now find the top 10 atricles from these filtered
filtered_df.groupby(['article_id']).count()['rating'].sort_values(ascending=False)[:10]

article_id
967     122
1425     91
467      90
2660     73
1562     71
456      69
221      64
1433     61
911      61
1904     61
Name: rating, dtype: int64

###### It has slightly different order of articles where no filter is applied but all are same

### Q2. Which are the most 10 read articles given that their average rating is above 1.5 

In [11]:
# Group the articles based on the mean of rating and then filter
filtered_df = df.groupby(['article_id']).mean().reset_index()
filtered_df[filtered_df['rating'] > 1.5].sort_values('rating', ascending=False)['article_id'][:10]

1636    1921
584      693
1870    2190
1385    1628
150      181
103      125
2017    2366
2369    2788
199      239
1214    1427
Name: article_id, dtype: int64

### Q3. Use the following formulation and identify the top 10 articles based on weighted rating as mentioned

In [12]:
# Mean of all ratings
C = np.mean(df['rating'])
# minimum num of ratings it set to default 2
m = 2
Cm = C*m

# create the dataframe which have the mean and number of ratings.
agg_df = df.groupby('article_id')['rating'].agg(R=np.mean, v='count').reset_index()

# function to calculate the weight against each article
def weight_index(row):
    return (row['R']*row['v'] + Cm) / (row['v'] + m)

# Apply the weight on each row
agg_df['W'] = agg_df.apply(weight_index, axis=1)

# Get the top 10 article by wieght
agg_df.sort_values(by='W', ascending=False)[['article_id', 'W']][:10]

,article_id,W
199,239,3.226974
1944,2283,3.226974
107,129,3.151316
1826,2141,3.151316
2363,2779,3.113487
789,931,3.090790
18,24,2.981579
617,739,2.981579
729,861,2.976974
1774,2079,2.863487
